In [1]:
%load_ext autoreload
%autoreload 2

In [90]:
import os
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient, OllamaClient

In [91]:
report_files = glob.glob("sample_reports/*.txt")

In [92]:
reports = []
report_ids = []
for report_file in report_files:
    report_ids.append(os.path.basename(report_file))
    with open(report_file, 'r') as f:
        reports.append(f.read())

In [95]:
# prompt = Prompt('demo_prompt_no_CoT.toml')
prompt = Prompt('sample_prompt.toml')

In [96]:
prompt

In [97]:
prompt.schemas[0]

In [98]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:8000/v1",
    temperature = 0.0,
    seed=42
)

In [99]:
# client = OllamaClient(
#     model = "phi3",
#     base_url = "http://localhost:11434/v1",
#     temperature = 0.0,
#     seed=42
# )

In [108]:
engine = RadPrompter(
    client=client,
    prompt=prompt,
    concurrency=12,
    hide_blocks=False,
    output_file="output3.csv",
)

In [109]:
sample = [{'report': sample_report, 'report_id': report_id} for sample_report, report_id in zip(reports, report_ids)]

In [110]:
sample

[{'report': 'Clinical Information:\n72-year-old female with sudden onset shortness of breath and hypoxia. Evaluate for pulmonary embolism.\nTechnique:\nCT pulmonary angiography with IV contrast was performed.\nFindings:\nThere is a large saddle embolus extending from the main pulmonary artery into the right and left main pulmonary arteries. Multiple filling defects are also seen within the segmental and subsegmental branches bilaterally, indicating additional pulmonary emboli.\nThe right ventricle appears dilated at 4.6 cm, greater than the left ventricle. There is flattening of the interventricular septum, suggesting right heart strain.\nMosaic perfusion is present within the lungs bilaterally, likely representing pulmonary infarcts.\nSmall bilateral pleural effusions are noted.\nImpression:\n\nSaddle pulmonary embolism involving the main pulmonary artery, right and left main branches, and multiple segmental/subsegmental branches.\nFindings of right heart strain with right ventricular

In [111]:
engine(sample)

Processing items:   0%|          | 0/3 [00:00<?, ?it/s]

Processing items: 100%|██████████| 3/3 [00:10<00:00,  3.40s/it]


In [112]:
results = pd.read_csv("output3.csv", index_col=0)
results

,Pulmonary Embolism_response_0,Pulmonary Embolism_response_1,Laterality_response_0,Laterality_response_1,Acuity_response_0,Acuity_response_1,report,report_id
index,,,,,,,,
1,Present\n</initial_answer>\n<explanation>\n1. ...,"""Present""\n",I'm going to take a guess that the initial ans...,"<json>\n{\n ""Laterality"" : ""Present""\n",I'm going to take a guess that the initial ans...,"<json>\n{\n ""Acuity"" : ""Not Mentioned""\n}\n",Here is an example radiology report describing...,sample_report_3.txt
2,Present\n</initial_answer>\n<explanation>\n1. ...,"""Present""\n",Bilateral\n</initial_answer>\n<explanation>\n1...,"<json>\n{\n ""Laterality"" : ""Bilateral""\n",I apologize for the mistake. Here is the corre...,"<json>\n{\n ""Acuity"" : ""Chronic""\n}\n",Clinical Information:\n67-year-old male with s...,sample_report_1.txt
0,Present\n</initial_answer>\n<explanation>\n1. ...,"""Present""\n",Bilateral\n</initial_answer>\n<explanation>\n1...,"<json>\n{\n ""Laterality"" : ""Bilateral""\n",<answer>\n<initial_answer>Acute\n</initial_ans...,"<json>\n{\n ""Acuity"" : ""Acute""\n}\n",Clinical Information:\n72-year-old female with...,sample_report_2.txt


In [52]:
engine.save_log("demo_no_CoT.log")